In [ ]:
import os
import io
from IPython.display import Image, display, HTML
import base64
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
hf_api_key = os.environ['HF_API_KEY']

We are using a diffusiom model for the above task

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="together",
    api_key="#############################",
)

# output is a PIL.Image object
image = client.text_to_image(
	"Astronaut riding a horse",
	model="stabilityai/stable-diffusion-xl-base-1.0",
)

In [ ]:
image.show()

In [ ]:
#making a function for the prompt
def generate_image(prompt):
    """Generate an image based on user input."""
    image = client.text_to_image(prompt, model="stabilityai/stable-diffusion-xl-base-1.0")
    image


In [ ]:
prompt = "An indian road with traffic"
generate_image(prompt)

In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(fn=generate_image,
                    inputs=[gr.Textbox(label="Your prompt")],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never",
                    examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

demo.launch(share=True)

In [ ]:
# generally we cant use the transformers pipeline for the stable diffusion models
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",cache_dir="D:\\.huggingface_cache\\huggingface\\hub")

In [ ]:
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipe(prompt).images[0]


In [ ]:
def generate(prompt):
    output = pipe(prompt)
    return output[0]['generated_text']


In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(fn=generate,
                    inputs=[gr.Textbox(label="Your prompt")],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never",
                    examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

demo.launch(share=True)

Using the gradio block

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Image Generation with Stable Diffusion")
    prompt = gr.Textbox(label="Your prompt")
    with gr.Row():
        with gr.Column():
            negative_prompt = gr.Textbox(label="Negative prompt")
            steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                      info="In many steps will the denoiser denoise the image?")
            guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
                      info="Controls how much the text prompt influences the result")
            width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
            height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
            btn = gr.Button("Submit")
        with gr.Column():
            output = gr.Image(label="Result")

    btn.click(fn=generate, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])
gr.close_all()
demo.launch(share=True)